purpose: to try running PCA analysis on the summary statistic level GWAS output for NICSA. This DID NOT WORK.

In [4]:
library(ggplot2)
library(dplyr)
library(reshape2)

In [5]:
getwd()

[1] "/tscc/projects/ps-palmer/brittany/rare_common_alcohol/notebooks"

In [6]:
setwd('/tscc/projects/ps-palmer/brittany/SUD_cross_species/')

In [7]:
#rerun removed regressedlr_nicsa_day1_infusion because it had a negative heritability estimate
#set true if want to use rerun LDSC values
rerun=FALSE

In [27]:
gwas_path='nicsa_gwas/mlma/mlma_concat/'

# import genetic correlation and heritability tables

In [8]:
nicsa_traits=read.table('scripts/nicsa_munge_ref/nicsa_traits.csv',sep=',',header=TRUE)

## GCTA- from GWAS pipeline- calculated by Thiago

In [9]:
herit_GCTA=read.table("nicsa_gwas/heritability.tsv",sep='\t',header=TRUE)
corr_tbl_GCTA=read.table('nicsa_gwas/genetic_correlation_melted_table.csv',sep=',',header=TRUE) #missing traits
corr_tbl_GCTA$trait1<-substring(corr_tbl_GCTA$trait1, first = 19)
corr_tbl_GCTA$trait2<-substring(corr_tbl_GCTA$trait2, first = 19)
corr_matrix_GCTA=read.table('nicsa_gwas/genetic_correlation_matrix.csv',sep=',',header=TRUE)
corr_matrix_gen_GCTA=read.table('nicsa_gwas/genetic_correlation_matrix_justgenetic.csv',sep=',',header=TRUE)

herit_GCTA columns

V.G.- general variance
V.e.- environmental variance
Vp- pheontypic variance (I think)
'V.G..Vp'- heritability
rG_SE-herit 
pval-probability of observing the estimated heritability under the NULL hypothesis (that the SNP heritability is 0)

In [10]:
gencor_GCTA=data.frame(corr_matrix_gen_GCTA)

In [11]:
gencor_GCTA

X,nicsa_day1_inactivelick,nicsa_day2_inactivelick,nicsa_day3_inactivelick,nicsa_day4_inactivelick,nicsa_total_inactivelick_10days,nicsa_day5_inactivelick,nicsa_day20_inactivelick,nicsa_day11_inactivelick,nicsa_day9_inactivelick,⋯,nicsa_day9_active_inactive_ratio,nicsa_day7_active_inactive_ratio,nicsa_day8_active_inactive_ratio,nicsa_day5_active_inactive_ratio,nicsa_day6_active_inactive_ratio,nicsa_active_inactive_ratio_all_days,nicsa_active_inactive_ratio_last_three_median,nicsa_day2_active_inactive_ratio,nicsa_day1_active_inactive_ratio,nicsa_active_inactive_ratio_first_three_median
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
nicsa_day1_inactivelick,1+-0,0.872392+-0.092099,0.491716+-0.116602,0.686206+-0.168172,0.765071+-0.073449,0.437478+-0.137482,0.497214+-0.145889,0.464856+-0.138153,0.362707+-0.135413,⋯,-0.42182+-0.126565,-0.382728+-0.130503,-0.38247+-0.136723,-0.417379+-0.138216,-0.501479+-0.124021,-0.455008+-0.103518,-0.403484+-0.128582,-0.371201+-0.154256,-0.257263+-0.142204,-0.251957+-0.137863
nicsa_day2_inactivelick,0.872392+-0.092099,1+-0,0.612603+-0.095302,0.783608+-0.147673,0.780538+-0.068914,0.491589+-0.124517,0.448365+-0.142433,0.352003+-0.138081,0.493698+-0.121845,⋯,-0.489231+-0.121141,-0.241391+-0.134147,-0.216925+-0.139105,-0.242589+-0.143849,-0.404205+-0.128095,-0.343149+-0.110346,-0.299498+-0.131037,-0.281345+-0.144117,-0.220502+-0.147311,-0.295507+-0.131348
nicsa_day3_inactivelick,0.491716+-0.116602,0.612603+-0.095302,1+-0,0.746094+-0.128626,0.66024+-0.072856,0.626861+-0.107149,0.576845+-0.129361,0.474597+-0.119001,0.660016+-0.105162,⋯,-0.188357+-0.122341,-0.221977+-0.12415,-0.177857+-0.127853,-0.106545+-0.13231,-0.251681+-0.124276,-0.367665+-0.099463,-0.257182+-0.120564,-0.368033+-0.1383,-0.38609+-0.131182,-0.357567+-0.117179
nicsa_day4_inactivelick,0.686206+-0.168172,0.783608+-0.147673,0.746094+-0.128626,1+-0,1.0+-0.073346,0.771184+-0.12092,0.747443+-0.188325,0.76176+-0.171891,0.857235+-0.15008,⋯,-0.446987+-0.19836,-0.308965+-0.193607,-0.411347+-0.211594,-0.4486+-0.205539,-0.530975+-0.199844,-0.363816+-0.156677,-0.461015+-0.190483,-0.575046+-0.215985,-0.411063+-0.208998,-0.450704+-0.19017
nicsa_total_inactivelick_10days,0.765071+-0.073449,0.780538+-0.068914,0.66024+-0.072856,1.0+-0.073346,1+-0,0.786973+-0.060571,0.682686+-0.097443,0.635784+-0.087642,0.803025+-0.05687,⋯,-0.501725+-0.101204,-0.387097+-0.106607,-0.498654+-0.10528,-0.435423+-0.114149,-0.505358+-0.100238,-0.436489+-0.085044,-0.314927+-0.107571,-0.479782+-0.121966,-0.318633+-0.122534,-0.395543+-0.110307
nicsa_day5_inactivelick,0.437478+-0.137482,0.491589+-0.124517,0.626861+-0.107149,0.771184+-0.12092,0.786973+-0.060571,1+-0,0.717897+-0.125771,0.64472+-0.121275,0.689758+-0.10804,⋯,-0.238659+-0.134098,-0.228841+-0.136903,-0.240146+-0.141518,-0.244061+-0.136166,-0.355775+-0.133504,-0.325001+-0.112996,-0.182765+-0.13494,-0.334602+-0.155837,-0.140534+-0.155394,-0.243785+-0.141788
nicsa_day20_inactivelick,0.497214+-0.145889,0.448365+-0.142433,0.576845+-0.129361,0.747443+-0.188325,0.682686+-0.097443,0.717897+-0.125771,1+-0,0.666357+-0.12777,0.621137+-0.12213,⋯,-0.234623+-0.141048,-0.1706+-0.145635,-0.241615+-0.145908,-0.265945+-0.150659,-0.440325+-0.137043,-0.384851+-0.120179,-0.358847+-0.134061,-0.436719+-0.164032,-0.325633+-0.156225,-0.299527+-0.149084
nicsa_day11_inactivelick,0.464856+-0.138153,0.352003+-0.138081,0.474597+-0.119001,0.76176+-0.171891,0.635784+-0.087642,0.64472+-0.121275,0.666357+-0.12777,1+-0,0.809908+-0.086074,⋯,-0.215352+-0.132143,-0.243832+-0.134221,-0.479732+-0.131063,-0.244543+-0.146054,-0.372363+-0.134747,-0.2542+-0.115792,-0.256405+-0.125723,-0.422215+-0.156819,-0.321882+-0.150151,-0.303389+-0.139809
nicsa_day9_inactivelick,0.362707+-0.135413,0.493698+-0.121845,0.660016+-0.105162,0.857235+-0.15008,0.803025+-0.05687,0.689758+-0.10804,0.621137+-0.12213,0.809908+-0.086074,1+-0,⋯,-0.395901+-0.116392,-0.452222+-0.124221,-0.547236+-0.122866,-0.490243+-0.13

## generate GCTA correlation matrix

In [12]:
corr_matrix_gen_GCTA=read.table('nicsa_gwas/genetic_correlation_matrix_justgenetic.csv',sep=',',header=TRUE)
corr_matrix_gen_GCTA<-data.frame(corr_matrix_gen_GCTA)
rownames(corr_matrix_gen_GCTA)<-corr_matrix_gen_GCTA$X
corr_matrix_gen_GCTA <- subset(corr_matrix_gen_GCTA, select = -c(X) )
colnames(corr_matrix_gen_GCTA)<-substring(colnames(corr_matrix_gen_GCTA), first = 7)
rownames(corr_matrix_gen_GCTA)<-substring(rownames(corr_matrix_gen_GCTA), first = 7)

In [13]:
corr_matrix_gen_GCTA<-corr_matrix_gen_GCTA[rownames(corr_matrix_gen_GCTA) %in% nicsa_traits$trait]
corr_matrix_gen_GCTA<-corr_matrix_gen_GCTA[colnames(corr_matrix_gen_GCTA) %in% nicsa_traits$trait]

In [15]:
#take only the value, remove the error, convert to numeric
corr_matrix_gen_GCTA <- corr_matrix_gen_GCTA %>% mutate(across(everything(), ~sapply(strsplit(., "\\+"), function(x) as.numeric(x[1]))))
#make into data matrix for melting
gencor_GCTA<-data.matrix(corr_matrix_gen_GCTA)
#melt matrix
gencor_GCTA_m<-melt(gencor_GCTA)

In [16]:
corr_matrix_smkinit<-corr_matrix_gen_GCTA[(rownames(corr_matrix_gen_GCTA) %in%  nicsa_traits[nicsa_traits$parallel=='smoking initiation',]$trait),]
corr_matrix_smkinit<-corr_matrix_smkinit[(colnames(corr_matrix_smkinit) %in%  nicsa_traits[nicsa_traits$parallel=='smoking initiation',]$trait)]

In [17]:
corr_matrix_cigday<-corr_matrix_gen_GCTA[(rownames(corr_matrix_gen_GCTA) %in%  nicsa_traits[nicsa_traits$parallel!='smoking initiation',]$trait),]
corr_matrix_cigday<-corr_matrix_cigday[(colnames(corr_matrix_cigday) %in%  nicsa_traits[nicsa_traits$parallel!='smoking initiation',]$trait)]

In [18]:
cormatrix<-corr_matrix_smkinit

In [20]:
eigenvalues<-round(eigen(cormatrix)$values,digits = 3)
explained_variance<-round(eigenvalues/sum(eigenvalues)*100,digits = 3)
library(knitr)
values<-cbind(c("1st PC","2nd PC","3rd PC","4th PC"),eigenvalues,explained_variance)
kable(values, caption="Eigenvalues and explained variance (%)", col.names = c("PC","Eigenvalues","Explained variance"))



Table: Eigenvalues and explained variance (%)

|PC     |Eigenvalues |Explained variance |
|:------|:-----------|:------------------|
|1st PC |6.152       |76.9               |
|2nd PC |1.105       |13.812             |
|3rd PC |0.535       |6.688              |
|4th PC |0.11        |1.375              |
|1st PC |0.075       |0.938              |
|2nd PC |0.027       |0.338              |
|3rd PC |0.011       |0.137              |
|4th PC |-0.015      |-0.188             |

In [ ]:
for f in os.listdir(gwas_path):
    if f.endswith('.mlma'):
        label=(f[18: (len(f)-5)])
            
        t=pd.read_csv(gwas_path+f,sep='\t')
        t.columns=['Chr', 'SNP', 'bp', 'A1', 'A2', 'Freq', 'b_'+label, 'se_'+label, 'p_'+label]
        if (label in list(nicsa_traits[nicsa_traits.parallel=='cigday'].trait)):
            print(label +' in cigday')
            if 'tbl_cigday' in globals():
                tbl_cigday=tbl_cigday.merge(t[['SNP','b_'+label, 'se_'+label, 'p_'+label]],on='SNP')
            else:
                tbl_cigday=t
        elif (label in list(nicsa_traits[nicsa_traits.parallel!='cigday'].trait)):
            print(label +' in smkinit')
            if 'tbl_smkinit' in globals():
                tbl_smkinit=tbl_smkinit.merge(t[['SNP','b_'+label, 'se_'+label, 'p_'+label]],on='SNP')
            else:
                tbl_smkinit=t
        else:
            print(label+' not in nicsa traits table')

active_inactive_ratio_first_three_median in smkinit


In [ ]:
files <- list.files(path = gwas_path, pattern = "\\.mlma$", full.names = TRUE)


In [89]:
gwama_path='nicsa_gwas/mlma/mlma_concat/GWAMA_format/'

In [71]:
# List files in the directory

# Initialize data frames
for (file in files[15:length(files)]) {
    label <- substr(basename(file), 19, nchar(basename(file)) - 5)
    print(label)
    # Read data
    t <- read.table(file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)
    colnames(t) <- c('CHR', 'SNPID', 'BP', 'EA', 'OA', 'EAF', 'Z','Z_SE', 'P')
    t$N<-(nicsa_traits[nicsa_traits$trait==label,]$N)
    fwrite(merged_file,file=paste0("gwama_",ref_trait), quote=FALSE,col.names=TRUE,row.names=F,sep=" ",na="NA")

    assign(label, t, envir = .GlobalEnv)}

[1] "last_three_days_activelick_total"
[1] "last_three_days_infusion_median"
[1] "last_three_days_infusion_total"
[1] "total_activelick_10days"
[1] "total_infusion_10days"


In [84]:
nicsa_traits$trait[1]

[1] "day1_activelick"

In [87]:
t<-get(nicsa_traits$trait[1], envir = .GlobalEnv)
fwrite(t,file=paste0("gwama_",ref_trait), quote=FALSE,col.names=TRUE,row.names=F,sep=" ",na="NA")


CHR,SNPID,BP,EA,OA,EAF,Z,Z_SE,P,N
<int>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1:735254,735254,A,G,0.927050,-0.02662060,0.0583923,0.6484670,2011
1,1:763121,763121,C,T,0.923536,-0.03152770,0.0599229,0.5987930,2011
1,1:767113,767113,A,G,0.923395,-0.03236390,0.0598346,0.5885840,2011
1,1:1113500,1113500,T,A,0.947046,-0.03809890,0.0719684,0.5965390,2011
1,1:1124362,1124362,G,A,0.947480,-0.05715170,0.0720771,0.4278220,2011
1,1:1169549,1169549,G,T,0.949241,-0.08898810,0.0725997,0.2202980,2011
1,1:1175836,1175836,G,A,0.949033,-0.09357690,0.0724579,0.1965420,2011
1,1:1181278,1181278,C,T,0.948798,-0.10315100,0.0722862,0.1535840,2011
1,1:1204576,1204576,C,A,0.947781,-0.10993500,0.0714223,0.1237490,2011


In [81]:
unique(active_inactive_ratio_first_three_median[active_inactive_ratio_first_three_median$P<1e-4,]$CHR)

[1]  1  2  5 14

In [39]:
(nicsa_traits[nicsa_traits$trait==label,]$N)

[1] 2018